# Keras Exercise

## Predict political party based on votes

As a fun little example, we'll use a public data set of how US congressmen voted on 17 different issues in the year 1984. Let's see if we can figure out their political party based on their votes alone, using a deep neural network!

For those outside the United States, our two main political parties are "Democrat" and "Republican." In modern times they represent progressive and conservative ideologies, respectively.

Politics in 1984 weren't quite as polarized as they are today, but you should still be able to get over 90% accuracy without much trouble.

Since the point of this exercise is implementing neural networks in Keras, I'll help you to load and prepare the data.

Let's start by importing the raw CSV file using Pandas, and make a DataFrame out of it with nice column labels:

In [4]:
import pandas as pd

feature_names =  ['party','handicapped-infants', 'water-project-cost-sharing', 
                    'adoption-of-the-budget-resolution', 'physician-fee-freeze',
                    'el-salvador-aid', 'religious-groups-in-schools',
                    'anti-satellite-test-ban', 'aid-to-nicaraguan-contras',
                    'mx-missle', 'immigration', 'synfuels-corporation-cutback',
                    'education-spending', 'superfund-right-to-sue', 'crime',
                    'duty-free-exports', 'export-administration-act-south-africa']

voting_data = pd.read_csv('house-votes-84.data.txt', na_values=['?'], 
                          names = feature_names)
voting_data.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


We can use describe() to get a feel of how the data looks in aggregate:

In [5]:
voting_data.describe()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
count,435,423,387,424,424,420,424,421,420,413,428,414,404,410,418,407,331
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,y,y,n,y,y,y,y,y,y,n,n,y,y,n,y
freq,267,236,195,253,247,212,272,239,242,207,216,264,233,209,248,233,269


We can see there's some missing data to deal with here; some politicians abstained on some votes, or just weren't present when the vote was taken. We will just drop the rows with missing data to keep it simple, but in practice you'd want to first make sure that doing so didn't introduce any sort of bias into your analysis (if one party abstains more than another, that could be problematic for example.)

In [6]:
voting_data.dropna(inplace=True)
voting_data.describe()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
count,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,n,y,n,y,y,y,y,n,y,n,n,y,y,n,y
freq,124,136,125,123,119,128,149,124,119,119,128,152,124,127,149,146,189


Our neural network needs normalized numbers, not strings, to work. So let's replace all the y's and n's with 1's and 0's, and represent the parties as 1's and 0's as well.

In [7]:
voting_data.replace(('y', 'n'), (1, 0), inplace=True)
voting_data.replace(('democrat', 'republican'), (1, 0), inplace=True)

In [8]:
voting_data.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
5,1,0,1,1,0,1,1,0,0,0,0,0,0,1,1,1,1
8,0,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
19,1,1,1,1,0,0,0,1,1,1,0,1,0,0,0,1,1
23,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,1,1
25,1,1,0,1,0,0,0,1,1,1,1,0,0,0,0,1,1


Finally let's extract the features and labels in the form that Keras will expect:

In [9]:
all_features = voting_data[feature_names].drop('party', axis=1).values
all_classes = voting_data['party'].values

OK, so have a go at it! You'll want to refer back to the slide on using Keras with binary classification - there are only two parties, so this is a binary problem. This also saves us the hassle of representing classes with "one-hot" format like we had to do with MNIST; our output is just a single 0 or 1 value.

Also refer to the scikit_learn integration slide, and use cross_val_score to evaluate your resulting model with 10-fold cross-validation.

**If you're using tensorflow-gpu on a Windows machine** by the way, you probably *do* want to peek a little bit at my solution - if you run into memory allocation errors, there's a workaround there you can use.

Try out your code here:

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import GridSearchCV

In [11]:
X_train, X_test, y_train, y_test = train_test_split(all_features, all_classes, test_size = 0.2)

In [12]:
def create_model():
    model = Sequential()
    model.add(Dense(16, activation='relu', input_shape=(16,)))
    model.add(Dropout(0.5))
    #model.add(Dense(10, activation='relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [13]:
estimator = KerasClassifier(build_fn=create_model, epochs=40, verbose=0)

c:\github\Private\private_env\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [14]:
cv_scores = cross_val_score(estimator, all_features, all_classes, cv=10)

In [15]:
print(cv_scores.mean())

0.9224637687206269


In [18]:
model = create_model()
history = model.fit(X_train, y_train,
                    batch_size=6,
                    epochs=40,
                    verbose=1,
                    validation_data=(X_test, y_test))

Epoch 1/40
31/31 [==============================] - 2s 19ms/step - loss: 0.8350 - accuracy: 0.3892 - val_loss: 0.6986 - val_accuracy: 0.4255
Epoch 2/40
31/31 [==============================] - 0s 5ms/step - loss: 0.7111 - accuracy: 0.5676 - val_loss: 0.6019 - val_accuracy: 0.7234
Epoch 3/40
31/31 [==============================] - 0s 5ms/step - loss: 0.6481 - accuracy: 0.5946 - val_loss: 0.5155 - val_accuracy: 0.8936
Epoch 4/40
31/31 [==============================] - 0s 4ms/step - loss: 0.5208 - accuracy: 0.7568 - val_loss: 0.4532 - val_accuracy: 0.8723
Epoch 5/40
31/31 [==============================] - 0s 4ms/step - loss: 0.4667 - accuracy: 0.8216 - val_loss: 0.4051 - val_accuracy: 0.8511
Epoch 6/40
31/31 [==============================] - 0s 4ms/step - loss: 0.4492 - accuracy: 0.8270 - val_loss: 0.3705 - val_accuracy: 0.8723
Epoch 7/40
31/31 [==============================] - 0s 4ms/step - loss: 0.4330 - accuracy: 0.8270 - val_loss: 0.3472 - val_accuracy: 0.8936
Epoch 8/40
31/31 [=

In [19]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.15067286789417267
Test accuracy: 0.957446813583374


In [20]:
predictions = model.predict(X_test).round().astype(int)
wrong = pd.DataFrame(columns=feature_names)
for i, (pred, act) in enumerate(zip(predictions, y_test)):
    if pred[0] != act:
        row_set = np.insert(X_test[i], obj=0, values=act)
        row = pd.DataFrame([row_set], columns=feature_names)
        wrong = wrong.append(row)

2/2 [==============================] - 0s 3ms/step


In [ ]:
wrong

VISUALIZE MODEL

In [21]:
from keras.utils.vis_utils import plot_model

In [22]:
plot_model(model, to_file='model_plot.png')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


GRID SEARCH

In [ ]:
def create_model_grid(activation = 'relu', 
                    optimizer = 'rmsprop', 
                    init='glorot_uniform'):
    model = Sequential()

    if activation == 'relu':
        model.add(Dense(16, activation='relu', input_shape=(16,)))
        model.add(Dense(1, activation='sigmoid'))
    if activation == 'tanh':
        model.add(Dense(16, activation='tanh', input_shape=(16,)))
        model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
model_grid = KerasClassifier(build_fn=create_model_grid, verbose=0)

In [ ]:
activations = ['relu', 'tanh']
optimizers = ['adam', 'rmsprop']
epochs = [10, 40, 60]
batches = [5, 10, 50]
param_grid = dict(activation = activations, 
    optimizer=optimizers, 
    epochs=epochs, 
    batch_size=batches,)

In [ ]:
grid = GridSearchCV(estimator=model_grid, param_grid=param_grid, cv = 3, verbose = 2)

In [ ]:
grid_result = grid.fit(all_features, all_classes)

In [ ]:
display(grid_result.best_params_)
display(grid_result.best_score_)


## My implementation is below

# No peeking!

![title](peek.jpg)

In [ ]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score

def create_model():
    model = Sequential()
    #16 feature inputs (votes) going into an 32-unit layer 
    model.add(Dense(32, input_dim=16, kernel_initializer='normal', activation='relu'))
    # Another hidden layer of 16 units
    model.add(Dense(16, kernel_initializer='normal', activation='relu'))
    # Output layer with a binary classification (Democrat or Republican political party)
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Wrap our Keras model in an estimator compatible with scikit_learn
estimator = KerasClassifier(build_fn=create_model, epochs=100, verbose=0)
# Now we can use scikit_learn's cross_val_score to evaluate this model identically to the others
cv_scores = cross_val_score(estimator, all_features, all_classes, cv=10)
cv_scores.mean()

94% without even trying too hard! Did you do better? Maybe more neurons, more layers, or Dropout layers would help even more.